# Create and run Earthquake Estimator

In [1]:
!nvidia-smi

Sun Mar 31 21:23:02 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:81:00.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import tensorflow as tf
import lstm_estimator
import earthquake_input_fn
import densenet
from importlib import reload
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os.path as osp
import pickle
%matplotlib inline

In [3]:
def reload_tf():
    global lstm_estimator
    global earthquake_input_fn
    global densenet
    global hooks
    for i in range(2):
        import lstm_estimator
        import earthquake_input_fn
        import hooks
        import densenet
        reload(lstm_estimator)
        reload(earthquake_input_fn)
        reload(hooks)
        reload(densenet)
        del lstm_estimator
        del earthquake_input_fn
        del hooks
        del densenet
    import lstm_estimator
    import earthquake_input_fn
    import densenet
    import hooks
reload_tf()

In [4]:
def create_densenet_from_params(batch_size, timesteps, time_pool, growth_rate, layer_sizes, dropout_rate, compression_theta,
                                dense_size, feature_columns, optimizer_name, learning_rate, lambda_l2_reg, grad_clip,
                                model_dir=None):
    
    params = {
        'batch_size': batch_size,
        'timesteps': timesteps,
        'time_pool': time_pool,
        'growth_rate': growth_rate,
        'layer_sizes': layer_sizes,
        'dropout_rate': dropout_rate,
        'compression_theta': compression_theta,
        'dense_size': dense_size,
        'feature_columns': feature_columns,
        'optimizer_name': optimizer_name,
        'learning_rate': learning_rate,
        'lambda_l2_reg': lambda_l2_reg,
        'grad_clip': grad_clip
    }
    
    if model_dir is None:
        model_dir = '/workspace/persistent-data/models/densenet-%s' % (time.strftime('%Y-%m-%d-%H-%M-%S'))
    
    params_file = osp.join(osp.split(model_dir)[0], osp.split(model_dir)[1] + '.params.pickle')
    if not osp.isfile(params_file):
        print('writing params to %s' % params_file)
        with open(params_file, 'wb') as f:
            pickle.dump(params, f)
        
    config = tf.estimator.RunConfig(model_dir=model_dir,
                                    log_step_count_steps=int(500 / batch_size),
                                    save_checkpoints_secs=300,
                                    session_config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.95)))
    
    estim = tf.estimator.Estimator(model_fn=densenet.densenet_model_fn,
                                   params=params,
                                   model_dir=model_dir,
                                   config=config)
    return estim

In [5]:
def train_and_evaluate_densenet(batch_size, timesteps, time_pool, growth_rate, layer_sizes, dropout_rate, compression_theta,
                                dense_size, feature_columns, optimizer_name, learning_rate, lambda_l2_reg, grad_clip,
                                noise,
                                earthquake_data_dir,
                                eval_count,
                                eval_every_n_secs,
                                model_dir=None):
    
    estim = create_densenet_from_params(batch_size=batch_size, timesteps=timesteps, time_pool=time_pool, 
                                        growth_rate=growth_rate, layer_sizes=layer_sizes, dropout_rate=dropout_rate,
                                        compression_theta=compression_theta, dense_size=dense_size,
                                        feature_columns=feature_columns, optimizer_name=optimizer_name,
                                        learning_rate=learning_rate, lambda_l2_reg=lambda_l2_reg, grad_clip=grad_clip,
                                        model_dir=model_dir)
    
    trainspec = tf.estimator.TrainSpec(input_fn=lambda: earthquake_input_fn.earthquake_input_fn2(earthquake_data_dir,
                                                                             batch_size,
                                                                             timesteps,
                                                                             noise,
                                                                             traintest='train',),
                                       max_steps=1000000)
    
    evalspec = tf.estimator.EvalSpec(input_fn=lambda: earthquake_input_fn.earthquake_input_fn2(earthquake_data_dir,
                                                                           batch_size,
                                                                           timesteps,
                                                                           noise,
                                                                           traintest='test'),
                                     steps=eval_count,
                                     start_delay_secs=eval_every_n_secs, throttle_secs=eval_every_n_secs)
    
    return estim, trainspec, evalspec

In [12]:
reload_tf()

EARTHQUAKE_DATA_DIR = '/workspace/persistent-data/earthquake/tfrecords5'
BATCH_SIZE = 32
TIMESTEPS = 150000
TIME_POOL = 125
GROWTH_RATE = 12
LAYER_SIZES = [6, 12, 24, 16]
DROPOUT_RATE = 0.4
COMPRESSION_THETA = 0.5
DENSE_SIZE=128
OPTIMIZER_NAME = 'RMSProp'
LEARNING_RATE = 0.001
LAMBDA_L2_REG = 0.0001
GRAD_CLIP = 1
NOISE = 0.00
EVAL_NUM_BATCHES = 100
EVAL_EVERY_N_SECONDS = 1800
FEATURE_COLUMNS = [tf.feature_column.numeric_column(key='acousticdata', dtype=tf.float32, shape=TIMESTEPS)]

estim, train_spec, eval_spec = train_and_evaluate_densenet(BATCH_SIZE, TIMESTEPS, TIME_POOL, GROWTH_RATE, LAYER_SIZES,
                                                           DROPOUT_RATE, COMPRESSION_THETA, DENSE_SIZE, FEATURE_COLUMNS,
                                                           OPTIMIZER_NAME, LEARNING_RATE, LAMBDA_L2_REG, GRAD_CLIP,
                                                           NOISE,
                                                           EARTHQUAKE_DATA_DIR,
                                                           EVAL_NUM_BATCHES,
                                                           EVAL_EVERY_N_SECONDS,
                                                           # MODEL_DIR
                                                          )

writing params to /workspace/persistent-data/models/densenet-2019-03-31-22-06-20.params.pickle
INFO:tensorflow:Using config: {'_service': None, '_log_step_count_steps': 15, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb48b3ae828>, '_save_checkpoints_secs': 300, '_experimental_distribute': None, '_num_worker_replicas': 1, '_num_ps_replicas': 0, '_model_dir': '/workspace/persistent-data/models/densenet-2019-03-31-22-06-20', '_eval_distribute': None, '_keep_checkpoint_max': 5, '_train_distribute': None, '_save_summary_steps': 100, '_tf_random_seed': None, '_master': '', '_device_fn': None, '_evaluation_master': '', '_task_id': 0, '_global_id_in_cluster': 0, '_protocol': None, '_task_type': 'worker', '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_is_chief': True}


In [ ]:
for i in range(5):
    try:
        print('---------------- BEGIN EPOCH %d ---------------' % i)
        tf.estimator.train_and_evaluate(estim, train_spec, eval_spec)
    except KeyboardInterrupt as e:
        raise e
    #except:
    #    print('end of epoch')

---------------- BEGIN EPOCH 0 ---------------
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 300.
INFO:tensorflow:Calling model_fn.
num_splits=1200 (this is the number of timesteps fed to stride CNN)
stft_input Tensor("input_ops/Cast_1:0", shape=(?, 1200, 65), dtype=float64)
stride_min Tensor("input_ops/Min:0", shape=(?, 1200), dtype=float64)
stride_max Tensor("input_ops/Max:0", shape=(?, 1200), dtype=float64)
stride_mean Tensor("input_ops/moments/Squeeze:0", shape=(?, 1200), dtype=float64)
stride_var Tensor("input_ops/moments/Squeeze_1:0", shape=(?, 1200), dtype=float64)
stride_pctiles Tensor("input_ops/concat:0", shape=(?, 1200, 6), dtype=float64)
stride_minroc Tensor("input_ops/Min_1:0", shape=(